# key for theyworkforyou API
DN8s9LBm8jMBFZihXEG2gqzx

In [1]:
#APIkey = 'DN8s9LBm8jMBFZihXEG2gqzx'
APIkey = 'AYC6jpECPBx7BjRRe3BEkMuR'

In [2]:
import requests
requests.get('https://www.theyworkforyou.com/api/getQuota?key=AYC6jpECPBx7BjRRe3BEkMuR')

<Response [200]>

In [3]:
requests.get('https://www.theyworkforyou.com/api/getQuota?key=%s' % APIkey, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}
).json()

{'quota': {'limit': 1000, 'current': 1036}}

In [4]:
def getQuota():
    quotaurl = 'https://www.theyworkforyou.com/api/getQuota'
    response = requests.get(quotaurl, params={'key':APIkey})
    return response.json()['quota']

In [5]:
getQuota()

{'limit': 1000, 'current': 1036}

In [6]:
postcodes_uk.validate('GL205FL')

NameError: name 'postcodes_uk' is not defined

In [8]:
import requests
import postcodes_uk

def is_valid_postcode(postcode):
    """
    This function takes one argument and returns True or False.
    
    Args:
    postcode (str): The postcode
    
    Returns:
    bool: True if a valid postcode, otherwise returns False.
    """
    #making sure the postcode is in uppercase, required for validation
    postcode = postcode.upper()
    # using the postcodes_uk library to validate the postcode
    ret = postcodes_uk.validate(postcode)
    return ret

def get_MP_info(postcode_or_constituency):
    """
    This function takes one argument, either the postcode or the constituency name, and returns the MP information from Hansard.
    
    Args:
    postcode_or_constituency (str): Postcode or a Constituency
    
    Returns:
    dict: The MP information from Hansard
    """
    # checks to see in the input is a valid postcode
    params = {'key':APIkey}
    if is_valid_postcode(postcode_or_constituency):
        params['postcode'] = postcode_or_constituency
    else:
        params['constituency'] = postcode_or_constituency
    
    url = 'https://www.theyworkforyou.com/api/getMP'
    response = requests.get(url, params=params)
    return response.json()

In [9]:
MP_info = get_MP_info('Tewkesbury')

In [10]:
MP_info

{'member_id': '42617',
 'house': '1',
 'constituency': 'Tewkesbury',
 'party': 'Conservative',
 'entered_house': '2019-12-13',
 'left_house': '9999-12-31',
 'entered_reason': 'general_election',
 'left_reason': 'still_in_office',
 'person_id': '10505',
 'lastupdate': '2019-12-13 05:21:12',
 'title': '',
 'given_name': 'Laurence',
 'family_name': 'Robertson',
 'full_name': 'Laurence Robertson',
 'url': '/mp/10505/laurence_robertson/tewkesbury',
 'image': '/people-images/mpsL/10505.jpg',
 'image_height': 160,
 'image_width': 120,
 'office': [{'moffice_id': 'uk.parliament.data/Member/253/Committee/11/1',
   'dept': 'Panel of Chairs',
   'position': 'Member',
   'from_date': '2020-01-15',
   'to_date': '9999-12-31',
   'person': '10505',
   'source': ''}]}

In [15]:
personID = MP_info['person_id']
params = {'key':APIkey}
params['person'] = personID
#params['num'] = "100"
url = 'https://www.theyworkforyou.com/api/getHansard'
response = requests.get(url, params=params)
response

<Response [200]>

In [16]:
personID

'10505'

In [20]:
from tqdm.notebook import tqdm
import functools

functools.lru_cache(maxsize = 128)
def get_Hansard(person_id):
    """
    This function takes one argument and returns the MP's person ID from Hansard.
    
    Args:
    personID (int): gets the relevant data from Hansard
    
    Returns:
    dict: The information from Hansard about the person_id
    """
    params = {'key':APIkey}
    params['person'] = person_id
    url = 'https://www.theyworkforyou.com/api/getHansard'
    response = requests.get(url, params=params)
    data = response.json()
    params['page'] = 1
    # loops until we have all the pages
    with tqdm(total=data['info']['total_results']) as pbar:
        pbar.update(len(data['rows']))
        while True:
            params['page'] += 1
            response = requests.get(url, params=params)
            data0 = response.json()
            pbar.update(len(data0['rows']))
            if len(data0['rows']) == 0:
                break
            data['rows'].extend(data0['rows'])
    return data

In [21]:
from bs4 import BeautifulSoup

def deduplicate(lst):
    deduplicated = []
    for item in lst:
        if item not in deduplicated:
            deduplicated.append(item)
    return deduplicated

def get_speeches(person_id):
    """
    This function takes one argument and returns the MP's person ID from Hansard.
    
    Args:
    person_id (int): gets the relevant data from Hansard
    
    Returns:
    list: Each list entry is one speech given by the MP
    """
    data = get_Hansard(person_id)
    speeches = []
    for x in data['rows']:
        body = x['body']
        text = BeautifulSoup(body, "html.parser").text
        speeches.append(text)
    return deduplicate(speeches)
        

In [39]:
MP_info = get_MP_info('M2 3GX')
speeches = get_speeches(MP_info['person_id'])

  0%|          | 0/2047 [00:00<?, ?it/s]

In [40]:
from sqlitedict import SqliteDict
db = SqliteDict("Hansard.sqlite")
db[MP_info['person_id']] = speeches
db.commit()

In [41]:
db.close()

In [42]:
from sqlitedict import SqliteDict
db = SqliteDict("Hansard.sqlite")
print(len(db))
db.close()

3


In [55]:
MP_info = get_MP_info('BN1 1FN')

In [56]:
db = SqliteDict("Hansard.sqlite")
D = {'info' : MP_info, 'speeches' : db[MP_info['person_id']]}
db[MP_info['person_id']] = D
db.commit()

In [57]:
db[MP_info['person_id']]

{'info': {'member_id': '42466',
  'house': '1',
  'constituency': 'Brighton, Pavilion',
  'party': 'Green',
  'entered_house': '2019-12-13',
  'left_house': '9999-12-31',
  'entered_reason': 'general_election',
  'left_reason': 'still_in_office',
  'person_id': '24910',
  'lastupdate': '2019-12-13 04:21:10',
  'title': '',
  'given_name': 'Caroline',
  'family_name': 'Lucas',
  'full_name': 'Caroline Lucas',
  'url': '/mp/24910/caroline_lucas/brighton%2C_pavilion',
  'image': '/people-images/mpsL/24910.jpg',
  'image_height': 160,
  'image_width': 120,
  'office': [{'moffice_id': 'uk.parliament.data/Member/3930/Committee/62/3',
    'dept': 'Environmental Audit Committee',
    'position': 'Member',
    'from_date': '2020-03-02',
    'to_date': '9999-12-31',
    'person': '24910',
    'source': ''}]},
 'speeches': ['uk',
  'To ask the Secretary of State for Foreign, Commonwealth and Development Affairs, at what stage under the ACRS pathway 3 process applicants and their dependants are fo

In [ ]:
import numpy as np
np.argmax([len(x['body']) for x in data['rows'][:128]])

In [ ]:
data['info']

In [ ]:


from bs4 import BeautifulSoup
BeautifulSoup(data['rows'][54]['body'], "html.parser").text



In [ ]:
from bs4 import BeautifulSoup
import string
extracts = [BeautifulSoup(x['body'], 'html.parser').text.lower().translate(str.maketrans('', '', string.punctuation)) for x in data['rows']]

In [ ]:
extracts[0]


In [ ]:
extracts = [e for e in extracts if len(e) > 512]

In [ ]:
100.0 * len([x for x in extracts if 'horse' in x]) / len(extracts)

In [ ]:
from wordcloud import WordCloud

In [ ]:
import matplotlib.pyplot as plt

# lower max_font_size
wc = WordCloud(max_font_size=40, background_color="black")
wc.stopwords = wc.stopwords.union({'speaker', 'secretary',
                                   'state', 'statement',
                                  'will', 'make', 'available'
                                   'made', 'ask', 'steps', 'b',
                                  'assessment', 'made', 'whether',
                                  'ensure', 'taking', 'take',
                                  'use', 'member', 'minister', 'friend', 'members', 'hon', 'mr','gentleman', 'lady', 'u', ';'})
wordcloud = wc.generate('\n'.join(extracts).lower())
plt.figure( figsize=(25,15) )
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")


In [ ]:
from rake_nltk import Rake
rake_nltk_var = Rake()
keywords = []
for e in extracts:
    rake_nltk_var.extract_keywords_from_text(e)
    keywords.append(rake_nltk_var.get_ranked_phrases()[:8])

In [ ]:
# lower max_font_size
wc = WordCloud(max_font_size=40, background_color="black")
wc.stopwords = wc.stopwords.union({'speaker', 'secretary',
                                   'state', 'statement',
                                  'will', 'make', 'available'
                                   'made', 'ask', 'steps', 'b',
                                  'assessment', 'made', 'whether',
                                  'ensure', 'taking', 'take',
                                  'use', 'member', 'minister', 'friend',
                                'members', 'hon', 'mr','gentleman',
                                 'lady', 'u', ';',
                                 'people', 'many'})
wordcloud = wc.generate(' '.join([' '.join(k) for k in keywords]))
plt.figure( figsize=(25,15) )
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
rake_nltk_var.get_ranked_phrases()[:8]

In [ ]:
K = ' '.join([' '.join(k) for k in keywords])

In [ ]:
K[:128]

In [ ]:
K.count(' ')